In [3]:
 #Download Libraries
!pip install azure-storage-blob


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00


In [4]:
#Import Libraries
import pandas as pd
import numpy as np
import json
import requests
import csv
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar

In [5]:

#Specify the path to your JSON configuration file
config_file_path= 'config.json'

#Load the JSON configuration file
with open (config_file_path, 'r') as config_file:
  config = json.load(config_file)

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'ownershippaymentdata'
#blob_name = ".csv"


# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string (CONNECTION_STRING_AZURE_STORAGE)
# Get the container client
container_client = blob_service_client.get_container_client (CONTAINER_AZURE)

ownershippaymentdata_df = pd.DataFrame()

# List all blobs in the speciffed container
blob_list = container_client.list_blobs ()
for blob in blob_list:
  print (blob.name)
  blob_client = container_client.get_blob_client (blob=blob.name)
  blob_data = blob_client.download_blob()
  blob_content = blob_data.readall().decode('utf-8')
  df = pd.read_csv(StringIO(blob_content))
  # Display the head of the DataFrame
  print (df.shape)
  # sind I have only one csv, I am doing to do the following instructions
  ownershippaymentdata_df = df.copy()

/content/OP_DTL_OWNRSHP_PGYR2022_P01182024 (1).csv
(3916, 30)


In [6]:
ownershippaymentdata_df.columns

Index(['Change_Type', 'Physician_Profile_ID', 'Physician_NPI',
       'Physician_First_Name', 'Physician_Middle_Name', 'Physician_Last_Name',
       'Physician_Name_Suffix',
       'Recipient_Primary_Business_Street_Address_Line1',
       'Recipient_Primary_Business_Street_Address_Line2', 'Recipient_City',
       'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country',
       'Recipient_Province', 'Recipient_Postal_Code', 'Physician_Primary_Type',
       'Physician_Specialty', 'Record_ID', 'Program_Year',
       'Total_Amount_Invested_USDollars', 'Value_of_Interest',
       'Terms_of_Interest',
       'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country',
       'Dispute_Status_for_Publicat

In [7]:
ownershippaymentdata_df.head()

,Change_Type,Physician_Profile_ID,Physician_NPI,Physician_First_Name,Physician_Middle_Name,Physician_Last_Name,Physician_Name_Suffix,Recipient_Primary_Business_Street_Address_Line1,Recipient_Primary_Business_Street_Address_Line2,Recipient_City,...,Value_of_Interest,Terms_of_Interest,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country,Dispute_Status_for_Publication,Interest_Held_by_Physician_or_an_Immediate_Family_Member,Payment_Publication_Date
0,UNCHANGED,835160,1.588622e+09,ELIZABETH,A,STEWART,NaN,200 1ST ST SW,CHARLTON 3A,ROCHESTER,...,28.13,Investment,"Gynesonics, Inc.",100000216788,"Gynesonics, Inc.",CA,United States,No,Physician Covered Recipient,01/18/2024
1,UNCHANGED,40572,1.891791e+09,Eric,NaN,Donnenfeld,NaN,711 Stewart Ave. Ste 160,NaN,Garden City,...,50000.00,Preferred Stock,"TearOptix, Inc.",100000971876,"TearOptix, Inc.",CA,United States,No,Physician Covered Recipient,01/18/2024
2,UNCHANGED,1175626,1.538509e+09,DAVID,GREGORY,WEINBERG,NaN,2776 CLEVELAND AVE,NaN,FORT MYERS,...,87162292.59,Investment Class A Common Units. Represents t...,MEDLINE INDUSTRIES LP,100000005431,MEDLINE INDUSTRIES LP,IL,United States,No,Immediate family member,01/18/2024
3,UNCHANGED,229524,1.073615e+09,JED,J,WEINBERG,NaN,200 WEST CARVER ST,NaN,HUNTINGTON,...,87162292.59,Investment Class A Common Units. Represents t...,MEDLINE INDUSTRIES LP,100000005431,MEDLINE INDUSTRIES LP,IL,United States,No,Immediate family member,01/18/2024
4,UNCHANGED,276414,1.104889e+09,Andrew,NaN,Beros,NaN,3811 Valley Centre Drive,NaN,San Diego,...,150935.00,Series A Preferred Stock; Series B Preferred S...,"Calyxo, Inc.",100000786821,"Calyxo, Inc.",CA,United States,No,Physician Covered Recipient,01/18/2024


In [8]:
ownershippaymentdata_df.drop(columns=["Physician_Middle_Name", "Physician_Name_Suffix", "Recipient_Province", "Recipient_Postal_Code"], inplace=True)

ownershippaymentdata_df.head()

,Change_Type,Physician_Profile_ID,Physician_NPI,Physician_First_Name,Physician_Last_Name,Recipient_Primary_Business_Street_Address_Line1,Recipient_Primary_Business_Street_Address_Line2,Recipient_City,Recipient_State,Recipient_Zip_Code,...,Value_of_Interest,Terms_of_Interest,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country,Dispute_Status_for_Publication,Interest_Held_by_Physician_or_an_Immediate_Family_Member,Payment_Publication_Date
0,UNCHANGED,835160,1.588622e+09,ELIZABETH,STEWART,200 1ST ST SW,CHARLTON 3A,ROCHESTER,MN,55905-0001,...,28.13,Investment,"Gynesonics, Inc.",100000216788,"Gynesonics, Inc.",CA,United States,No,Physician Covered Recipient,01/18/2024
1,UNCHANGED,40572,1.891791e+09,Eric,Donnenfeld,711 Stewart Ave. Ste 160,NaN,Garden City,NY,11530,...,50000.00,Preferred Stock,"TearOptix, Inc.",100000971876,"TearOptix, Inc.",CA,United States,No,Physician Covered Recipient,01/18/2024
2,UNCHANGED,1175626,1.538509e+09,DAVID,WEINBERG,2776 CLEVELAND AVE,NaN,FORT MYERS,FL,33901-5864,...,87162292.59,Investment Class A Common Units. Represents t...,MEDLINE INDUSTRIES LP,100000005431,MEDLINE INDUSTRIES LP,IL,United States,No,Immediate family member,01/18/2024
3,UNCHANGED,229524,1.073615e+09,JED,WEINBERG,200 WEST CARVER ST,NaN,HUNTINGTON,NY,11743,...,87162292.59,Investment Class A Common Units. Represents t...,MEDLINE INDUSTRIES LP,100000005431,MEDLINE INDUSTRIES LP,IL,United States,No,Immediate family member,01/18/2024
4,UNCHANGED,276414,1.104889e+09,Andrew,Beros,3811 Valley Centre Drive,NaN,San Diego,CA,92130,...,150935.00,Series A Preferred Stock; Series B Preferred S...,"Calyxo, Inc.",100000786821,"Calyxo, Inc.",CA,United States,No,Physician Covered Recipient,01/18/2024


In [9]:
# Mapping dictionary
change_type_dict = {
    'NEW': 'Records added from the end of the previous submission deadline',
    'ADDED': 'Records that were not eligible at the time of previous publication, which is eligible for current publication',
    'CHANGED': 'Previously published records modified after the last publication',
    'UNCHANGED': 'Previously published records that remain “unchanged” in current publication'
}
unique_change_type = ownershippaymentdata_df ['Change_Type'].unique()
# Converting the array of unique values into a DataFrame
unique_change_type_df = pd.DataFrame(unique_change_type, columns=['Record_ID'])
# Applying the mapping to create a new column with descriptions
unique_change_type_df ['Change_Type'] = unique_change_type_df['Record_ID'].map(change_type_dict)
unique_change_type_df = unique_change_type_df [unique_change_type_df['Record_ID'] != '<NA>']

unique_change_type_df

,Record_ID,Change_Type
0,UNCHANGED,Previously published records that remain “unch...
1,CHANGED,Previously published records modified after th...
2,NEW,Records added from the end of the previous sub...


In [10]:
Dim_Physician = df[['Physician_Profile_ID',
                             'Physician_First_Name', 'Physician_Last_Name',
                             'Physician_Primary_Type', 'Physician_Specialty']].drop_duplicates()

In [11]:
Dim_Recipient = df[['Recipient_Primary_Business_Street_Address_Line1',
                             'Recipient_City', 'Recipient_State',
                             'Recipient_Zip_Code', 'Recipient_Country']].drop_duplicates()

In [12]:
Dim_Payment_Details = df[['Record_ID',
                             'Program_Year']].drop_duplicates()

In [17]:
fact_ownership_payment = df[['Record_ID','Physician_Profile_ID', 'Total_Amount_Invested_USDollars', 'Value_of_Interest', 'Terms_of_Interest', 'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country', 'Dispute_Status_for_Publication', 'Interest_Held_by_Physician_or_an_Immediate_Family_Member', 'Payment_Publication_Date']].drop_duplicates()


In [18]:
fact_ownership_payment.head()

,Record_ID,Physician_Profile_ID,Total_Amount_Invested_USDollars,Value_of_Interest,Terms_of_Interest,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country,Dispute_Status_for_Publication,Interest_Held_by_Physician_or_an_Immediate_Family_Member,Payment_Publication_Date
0,957788543,835160,0.0,28.13,Investment,"Gynesonics, Inc.",100000216788,"Gynesonics, Inc.",CA,United States,No,Physician Covered Recipient,01/18/2024
1,957789753,40572,50000.0,50000.00,Preferred Stock,"TearOptix, Inc.",100000971876,"TearOptix, Inc.",CA,United States,No,Physician Covered Recipient,01/18/2024
2,957800853,1175626,0.0,87162292.59,Investment Class A Common Units. Represents t...,MEDLINE INDUSTRIES LP,100000005431,MEDLINE INDUSTRIES LP,IL,United States,No,Immediate family member,01/18/2024
3,957800855,229524,0.0,87162292.59,Investment Class A Common Units. Represents t...,MEDLINE INDUSTRIES LP,100000005431,MEDLINE INDUSTRIES LP,IL,United States,No,Immediate family member,01/18/2024
4,957887773,276414,0.0,150935.00,Series A Preferred Stock; Series B Preferred S...,"Calyxo, Inc.",100000786821,"Calyxo, Inc.",CA,United States,No,Physician Covered Recipient,01/18/2024
